# Training model LLM use Unsloth

In [1]:
!nvidia-smi

Wed Dec  3 23:54:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.105.08             Driver Version: 580.105.08     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:07:00.0 Off |                    0 |
| N/A   54C    P0             27W /   70W |    6257MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

#### Cài đặt thư viện 

In [2]:
# Cell 1: Cài đặt các thư viện cần thiết
# Unsloth: Framework tối ưu cho việc finetune LLM models nhanh hơn và tiết kiệm memory
# xformers: Tối ưu hóa transformer operations
# trl: Transformer Reinforcement Learning - thư viện cho supervised fine-tuning
# peft: Parameter-Efficient Fine-Tuning - chỉ train một phần nhỏ parameters
# accelerate: Tăng tốc training trên multi-GPU
# bitsandbytes: Quantization library để giảm memory footprint

! pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
! pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-_aimmuky/unsloth_e6a29bbabafd4650adfccb523cbabbef
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-_aimmuky/unsloth_e6a29bbabafd4650adfccb523cbabbef
  Resolved https://github.com/unslothai/unsloth.git to commit 6789c279d578278aca4af22f4ca31fc42829c9a4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached xformers-0.0.26.post1.tar.gz (4.1 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      Traceback (most recent call last):
        File "/home/thanh/miniconda3/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 389, in <module>

### Import thư viện và load data

In [ ]:
import json
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import Dataset

# Load dữ liệu từ file JSON
with open('data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Kiểm tra số lượng samples
print(f"Tổng số câu hỏi-trả lời: {len(data)}")
print(f"Ví dụ đầu tiên: {data[0]}")

# Convert sang Hugging Face Dataset format để dễ xử lý
dataset = Dataset.from_list(data)
print(f"\nDataset info: {dataset}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/thanh/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
Tổng số câu hỏi-trả lời: 199
Ví dụ đầu tiên: {'instruction': 'Bạn tên là gì?', 'input': '', 'output': 'Tôi tên là Nguyễn Tuấn Thanh, một Backend Engineer và AI Engineer đang sinh sống tại Cầu Giấy, Hà Nội.'}

Dataset info: Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 199
})


### Cấu hình và load model

In [4]:
# max_seq_length: Độ dài tối đa của sequence (input + output)
# - Tăng lên nếu câu trả lời dài, nhưng tốn memory nhiều hơn
# - 2048 tokens ≈ 1500 words tiếng Anh hoặc tương đương
max_seq_length = 2048

# dtype: Kiểu dữ liệu cho model weights
# - None: Auto-detect (Float16 cho GPU cũ, Bfloat16 cho GPU mới)
# - Float16: Tiết kiệm memory nhưng có thể bị numerical instability
# - Bfloat16: Tốt hơn cho training stability (cần GPU Ampere trở lên)
dtype = None  

# load_in_4bit: Quantization 4-bit
# - True: Giảm memory usage xuống ~4x (12GB model → 3GB)
# - Cho phép train model lớn trên GPU nhỏ
# - Trade-off: Giảm nhẹ accuracy nhưng thường không đáng kể
load_in_4bit = True  

print("Đang load model Gemma-2-9B...")
print("⏳ Quá trình này có thể mất 2-5 phút...")

# Load pre-trained model và tokenizer
# unsloth/gemma-2-9b-bnb-4bit: Gemma-2-9B đã được quantize 4-bit sẵn
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gemma-2-9b-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

print("✅ Model loaded successfully!")
print(f"Model size: {sum(p.numel() for p in model.parameters()) / 1e9:.2f}B parameters")

Đang load model Gemma-2-9B...
⏳ Quá trình này có thể mất 2-5 phút...
==((====))==  Unsloth 2025.11.6: Fast Gemma2 patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
✅ Model loaded successfully!
Model size: 5.08B parameters


###  Thêm LoRA adapters

In [5]:
# LoRA là kỹ thuật PEFT - chỉ train thêm một số parameters nhỏ thay vì toàn bộ model

print("Đang thêm LoRA adapters...")

model = FastLanguageModel.get_peft_model(
    model,
    
    # r: Rank của LoRA matrices (low-rank decomposition)
    # - Giá trị nhỏ hơn → ít parameters hơn → train nhanh hơn nhưng capacity thấp
    # - Giá trị lớn hơn → nhiều parameters → học được nhiều hơn nhưng train lâu
    # - Recommended: 8-64 (16 là sweet spot cho most cases)
    r=16,
    
    # target_modules: Các layers sẽ apply LoRA
    # - q_proj, k_proj, v_proj: Query, Key, Value projections trong attention
    # - o_proj: Output projection của attention
    # - gate_proj, up_proj, down_proj: MLP layers
    # - Train càng nhiều modules → học được nhiều hơn nhưng tốn memory
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    
    # lora_alpha: Scaling factor cho LoRA weights
    # - Thường set bằng r hoặc 2*r
    # - Cao hơn → LoRA có impact mạnh hơn lên model
    lora_alpha=16,
    
    # lora_dropout: Dropout rate cho LoRA layers
    # - 0: Không dropout → train nhanh nhất (optimized by Unsloth)
    # - 0.05-0.1: Giúp prevent overfitting nếu dataset nhỏ
    lora_dropout=0,
    
    # bias: Có train bias terms không
    # - "none": Không train bias (fastest, recommended)
    # - "all": Train tất cả biases
    # - "lora_only": Chỉ train biases của LoRA layers
    bias="none",
    
    # use_gradient_checkpointing: Tiết kiệm memory bằng cách recompute
    # - "unsloth": Unsloth's optimized checkpointing (recommended)
    # - True: Standard gradient checkpointing
    # - False: Không dùng (nhanh nhưng tốn memory)
    use_gradient_checkpointing="unsloth",
    
    # random_state: Seed cho reproducibility
    random_state=3407,
    
    # use_rslora: Rank-Stabilized LoRA
    # - False: Standard LoRA (recommended for most cases)
    # - True: Better stability với high rank values
    use_rslora=False,
    
    # loftq_config: LoftQ initialization
    # - None: Standard initialization (recommended)
    # - Config object: Special init for quantized models
    loftq_config=None,
)

# In ra số lượng trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"✅ LoRA adapters added!")
print(f"Trainable parameters: {trainable_params:,} ({100 * trainable_params / total_params:.2f}%)")
print(f"Total parameters: {total_params:,}")

Đang thêm LoRA adapters...


Unsloth 2025.11.6 patched 42 layers with 42 QKV layers, 42 O layers and 42 MLP layers.


✅ LoRA adapters added!
Trainable parameters: 54,018,048 (1.05%)
Total parameters: 5,133,925,888


### Format dữ liệu

In [6]:
# Cell 5: Format data cho training
# Tạo prompt template phù hợp với task Q&A về bạn

# EOS_TOKEN: End of Sequence token - đánh dấu kết thúc câu trả lời
# CRITICAL: Phải thêm EOS_TOKEN để model biết khi nào dừng generate
EOS_TOKEN = tokenizer.eos_token

# Template cho prompt - customize theo style bạn muốn
# Format: Instruction-based prompting
alpaca_prompt = """Dưới đây là một câu hỏi về Nguyễn Tuấn Thanh. Hãy trả lời chính xác và chi tiết.

### Câu hỏi:
{}

### Ngữ cảnh thêm:
{}

### Trả lời:
{}"""

def formatting_prompts_func(examples):
    """
    Hàm format data thành prompt strings
    
    Args:
        examples: Dict chứa các lists của instruction, input, output
    
    Returns:
        Dict với key "text" chứa formatted prompts
    """
    instructions = examples["instruction"]  # Câu hỏi
    inputs = examples["input"]              # Context thêm (có thể rỗng)
    outputs = examples["output"]            # Câu trả lời
    texts = []
    
    for instruction, inp, output in zip(instructions, inputs, outputs):
        # Format theo template và thêm EOS_TOKEN
        # EOS_TOKEN giúp model biết khi nào stop generating
        text = alpaca_prompt.format(instruction, inp, output) + EOS_TOKEN
        texts.append(text)
    
    return {"text": texts}

# Apply formatting function lên toàn bộ dataset
# batched=True: Xử lý nhiều examples cùng lúc (nhanh hơn)
print("Đang format dataset...")
dataset = dataset.map(formatting_prompts_func, batched=True)

print("✅ Dataset formatted!")
print(f"Sample formatted text:\n{dataset[0]['text'][:500]}...")

Đang format dataset...


Map: 100%|██████████| 199/199 [00:00<00:00, 9485.82 examples/s]

✅ Dataset formatted!
Sample formatted text:
Dưới đây là một câu hỏi về Nguyễn Tuấn Thanh. Hãy trả lời chính xác và chi tiết.

### Câu hỏi:
Bạn tên là gì?

### Ngữ cảnh thêm:


### Trả lời:
Tôi tên là Nguyễn Tuấn Thanh, một Backend Engineer và AI Engineer đang sinh sống tại Cầu Giấy, Hà Nội.<eos>...


### Cấu hình training arguments

In [7]:
training_args = TrainingArguments(
    # per_device_train_batch_size: Số samples per GPU per step
    # - Tăng lên để train nhanh hơn (nếu đủ memory)
    # - Giảm xuống nếu bị OOM (Out of Memory)
    # - Recommended: 2-8 cho 4-bit quantized models
    per_device_train_batch_size=2,
    
    # gradient_accumulation_steps: Tích lũy gradients qua N steps
    # - Effective batch size = per_device_batch_size * gradient_accumulation_steps * num_gpus
    # - Tăng lên để simulate batch size lớn hơn mà không tốn memory
    # - Ví dụ: 2 * 4 = effective batch size 8
    gradient_accumulation_steps=4,
    
    # warmup_steps: Số steps để tăng learning rate từ 0 lên max
    # - Giúp training stable hơn ở đầu
    # - Recommended: 5-10% total steps
    # - Ví dụ: 5-10 steps cho small training runs
    warmup_steps=10,
    
    # max_steps: Tổng số training steps
    # - 1 step = 1 forward + 1 backward pass
    # - Tăng lên để train lâu hơn (better results nhưng risk overfit)
    # - Recommended: 100-500 steps cho dataset 300 samples
    # - Set -1 để train theo num_train_epochs thay vì steps
    max_steps=200,  # Tăng từ 60 lên 200 để học tốt hơn
    
    # learning_rate: Tốc độ học (quan trọng nhất!)
    # - Quá lớn → model không converge, loss oscillate
    # - Quá nhỏ → học chậm, có thể stuck
    # - Recommended cho LoRA: 1e-4 đến 5e-4
    # - 2e-4 là good default
    learning_rate=2e-4,
    
    # fp16 / bf16: Mixed precision training
    # - fp16: Dùng cho GPU cũ (Tesla T4, V100)
    # - bf16: Dùng cho GPU mới (A100, H100, RTX 30/40 series)
    # - Giảm memory usage và tăng speed ~2x
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    
    # logging_steps: Log metrics sau mỗi N steps
    # - 1: Log mọi step (chi tiết nhất)
    # - 10: Log mỗi 10 steps (ít spam hơn)
    logging_steps=5,
    
    # optim: Optimizer algorithm
    # - "adamw_8bit": AdamW với 8-bit quantization (tiết kiệm memory)
    # - "adamw_torch": Standard AdamW
    # - "sgd": Stochastic Gradient Descent (ít dùng)
    optim="adamw_8bit",
    
    # weight_decay: L2 regularization
    # - Giúp prevent overfitting
    # - Recommended: 0.01-0.1
    # - 0: Không regularization
    weight_decay=0.01,
    
    # lr_scheduler_type: Learning rate schedule
    # - "linear": Giảm dần từ max xuống 0 (recommended)
    # - "cosine": Giảm theo cosine curve
    # - "constant": Giữ nguyên learning rate
    lr_scheduler_type="linear",
    
    # seed: Random seed cho reproducibility
    # - Set giống nhau để results giống nhau mỗi run
    seed=3407,
    
    # output_dir: Thư mục save checkpoints
    output_dir="outputs",
    
    # report_to: Nơi log metrics
    # - "none": Không log external (chỉ console)
    # - "wandb": Log lên Weights & Biases
    # - "tensorboard": Log lên TensorBoard
    report_to="none",
    
    # save_strategy: Khi nào save checkpoint
    # - "steps": Save mỗi save_steps
    # - "epoch": Save cuối mỗi epoch
    # - "no": Không save (chỉ save final)
    save_strategy="steps",
    save_steps=50,  # Save mỗi 50 steps
    
    # eval_strategy: Khi nào evaluate (nếu có eval dataset)
    # - "steps": Eval mỗi eval_steps
    # - "epoch": Eval cuối epoch
    # - "no": Không eval
    # evaluation_strategy="steps",
    # eval_steps=50,
)

print("✅ Training arguments configured!")
print(f"Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"Total training steps: {training_args.max_steps}")
print(f"Learning rate: {training_args.learning_rate}")

✅ Training arguments configured!
Effective batch size: 8
Total training steps: 200
Learning rate: 0.0002


### Khởi tạo Trainer

In [8]:
trainer = SFTTrainer(
    # model: Model với LoRA adapters
    model=model,
    
    # tokenizer: Để tokenize text thành numbers
    tokenizer=tokenizer,
    
    # train_dataset: Dataset đã format
    train_dataset=dataset,
    
    # dataset_text_field: Tên column chứa text đã format
    # - Phải match với output của formatting_prompts_func
    dataset_text_field="text",
    
    # max_seq_length: Max length từ model config
    max_seq_length=max_seq_length,
    
    # dataset_num_proc: Số processes cho data processing
    # - Tăng lên để preprocess nhanh hơn (nếu có nhiều CPU cores)
    # - 2-4 là reasonable
    dataset_num_proc=2,
    
    # packing: Pack nhiều short sequences vào một sequence
    # - True: Hiệu quả hơn cho short texts (tăng throughput 5x)
    # - False: An toàn hơn, không mix different conversations
    # - Recommended: False cho Q&A tasks để tránh confusion
    packing=False,
    
    # args: Training arguments từ cell trước
    args=training_args,
)

print("✅ Trainer initialized!")
print(f"Number of training samples: {len(dataset)}")
print(f"Estimated training time: ~{training_args.max_steps * 2 / 60:.1f} minutes")

Unsloth: Tokenizing ["text"] (num_proc=12): 100%|██████████| 199/199 [00:10<00:00, 19.52 examples/s]

✅ Trainer initialized!
Number of training samples: 199
Estimated training time: ~6.7 minutes


### Bắt đầu training

In [9]:
print("="*50)
print("🚀 BẮT ĐẦU TRAINING!")
print("="*50)
print(f"Dataset size: {len(dataset)} samples")
print(f"Training steps: {training_args.max_steps}")
print(f"Batch size: {training_args.per_device_train_batch_size}")
print(f"Learning rate: {training_args.learning_rate}")
print("="*50)
print("\n⏳ Training in progress... Hãy kiên nhẫn!\n")

# Train!
# Quá trình này sẽ:
# 1. Load batches of data
# 2. Forward pass (tính predictions)
# 3. Compute loss (sai số giữa prediction và actual)
# 4. Backward pass (tính gradients)
# 5. Update weights
# 6. Repeat

trainer_stats = trainer.train()

print("\n" + "="*50)
print("✅ TRAINING HOÀN TẤT!")
print("="*50)
print(f"Final loss: {trainer_stats.training_loss:.4f}")
print(f"Training time: {trainer_stats.metrics['train_runtime']:.2f} seconds")
print(f"Samples per second: {trainer_stats.metrics['train_samples_per_second']:.2f}")
print("="*50)

The model is already on multiple devices. Skipping the move to device specified in `args`.


🚀 BẮT ĐẦU TRAINING!
Dataset size: 199 samples
Training steps: 200
Batch size: 2
Learning rate: 0.0002

⏳ Training in progress... Hãy kiên nhẫn!



==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 199 | Num Epochs = 8 | Total steps = 200
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 54,018,048 of 9,295,724,032 (0.58% trained)


Step,Training Loss
5,3.316000
10,1.825300
15,1.191400
20,0.970400
25,0.921100
30,0.663500
35,0.667400
40,0.658000
45,0.634100
50,0.571600


Unsloth: Will smartly offload gradients to save VRAM!


'(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: 55fb8e21-363d-406b-95cf-bbd5e63026d9)')' thrown while requesting HEAD https://huggingface.co/unsloth/gemma-2-9b-bnb-4bit/resolve/main/config.json
[huggingface_hub.utils._http|WARNING]'(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: 55fb8e21-363d-406b-95cf-bbd5e63026d9)')' thrown while requesting HEAD https://huggingface.co/unsloth/gemma-2-9b-bnb-4bit/resolve/main/config.json
Retrying in 1s [Retry 1/5].
[huggingface_hub.utils._http|WARNING]Retrying in 1s [Retry 1/5].
'(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: 123357fc-2e2a-4220-bc3f-dbeae703d618)')' thrown while requesting HEAD https://huggingface.co/unsloth/gemma-2-9b-bnb-4bit/resolve/main/config.json
[huggingface_hub.utils._http|WARNING]'(ProtocolError('Connectio


✅ TRAINING HOÀN TẤT!
Final loss: 0.3948
Training time: 1151.40 seconds
Samples per second: 1.39


### Test model

In [17]:
print("Đang test model...")

# Enable inference mode (faster, no gradient computation)
FastLanguageModel.for_inference(model)

# Danh sách câu hỏi test
test_questions = [
    # "Bạn tên là gì?",
    # "Bạn làm việc ở đâu?",
    # "Kỹ năng của bạn là gì?",
    # "Dự án nào bạn tự hào nhất?",
    # "Bạn học trường nào?"
    "Bạn hãy giới thiệu các thông tin về bạn đi nhớ giới thiệu kĩ và dài vào bao gồm sở thích, các dự án đã làm, kĩ năng đã tích lũy."
]

print("\n" + "="*50)
print("🧪 TESTING MODEL")
print("="*50)

for question in test_questions:
    # Format theo template
    prompt = alpaca_prompt.format(
        question,  # instruction
        "",        # input (empty)
        ""         # output (để model generate)
    )
    
    # Tokenize input
    inputs = tokenizer(
        [prompt], 
        return_tensors="pt",
        padding=True,
        truncation=True
    ).to("cuda")
    
    # Generate response
    # max_new_tokens: Số tokens tối đa cho response
    # temperature: 0.7 = creative hơn, 0.1 = deterministic hơn
    # top_p: Nucleus sampling (0.9 là good default)
    # do_sample: True = sample từ distribution, False = greedy (chọn highest prob)
    outputs = model.generate(
        **inputs, 
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        use_cache=True  # Cache key/value trong attention để generate nhanh hơn
    )
    
    # Decode output
    generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    
    # Extract only the response part (after "### Trả lời:")
    response = generated_text.split("### Trả lời:")[-1].strip()
    
    print(f"\n❓ Câu hỏi: {question}")
    print(f"💬 Trả lời: {response}")
    print("-"*50)

print("\n✅ Testing complete!")

Đang test model...

🧪 TESTING MODEL

❓ Câu hỏi: Bạn hãy giới thiệu các thông tin về bạn đi nhớ giới thiệu kĩ và dài vào bao gồm sở thích, các dự án đã làm, kĩ năng đã tích lũy.
💬 Trả lời: Tôi tên là Nguyễn Tuấn Thanh, một Backend Engineer và AI Engineer đang sinh sống tại Cầu Giấy, Hà Nội. Sở thích của tôi là research AI/ML papers mới, contribute to open-source projects, và build scalable backend systems.
--------------------------------------------------

✅ Testing complete!


### Lưu model

In [11]:
print("Đang lưu model...")

# Option 1: Lưu chỉ LoRA adapters (nhẹ nhất, ~10-50MB)
# Khi load lại cần base model + adapters này
model.save_pretrained("gemma2_thanh_lora")
tokenizer.save_pretrained("gemma2_thanh_lora")
print("✅ Saved LoRA adapters to gemma2_thanh_lora/")

# Option 2: Lưu merged model (base + LoRA) ở 16-bit
# File size lớn hơn (~18GB) nhưng dễ dùng hơn
# model.save_pretrained_merged(
#     "gemma2_thanh_merged_16bit",
#     tokenizer,
#     save_method="merged_16bit"
# )
# print("✅ Saved merged 16-bit model to gemma2_thanh_merged_16bit/")

# Option 3: Lưu merged + quantized về 4-bit (balance size và quality)
# model.save_pretrained_merged(
#     "gemma2_thanh_merged_4bit",
#     tokenizer,
#     save_method="merged_4bit"
# )
# print("✅ Saved merged 4-bit model to gemma2_thanh_merged_4bit/")

print("\n📁 Files saved:")
print("  - adapter_config.json: LoRA configuration")
print("  - adapter_model.safetensors: LoRA weights")
print("  - tokenizer files: Để encode/decode text")

Đang lưu model...
✅ Saved LoRA adapters to gemma2_thanh_lora/

📁 Files saved:
  - adapter_config.json: LoRA configuration
  - adapter_model.safetensors: LoRA weights
  - tokenizer files: Để encode/decode text


### Push to Hugging Face Hub

In [ ]:
# Cần Hugging Face account và access token

# Uncomment và fill in your info để push
"""
from huggingface_hub import login

# Login to HuggingFace
# Get token from: https://huggingface.co/settings/tokens
login(token="YOUR_HF_TOKEN_HERE")

# Push LoRA adapters
model.push_to_hub(
    "ntthanh2603/gemma2-thanh-lora",  # Repo name
    token="YOUR_HF_TOKEN_HERE",
    private=False  # False = public, True = private
)
tokenizer.push_to_hub(
    "ntthanh2603/gemma2-thanh-lora",
    token="YOUR_HF_TOKEN_HERE"
)

print("Model pushed to HuggingFace Hub!")
print("View at: https://huggingface.co/ntthanh2603/gemma2-thanh-lora")
"""

### Load model đã save để dùng lại

In [ ]:
"""
# Khi muốn load model đã save:

from unsloth import FastLanguageModel

# Load base model + LoRA adapters
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="gemma2_thanh_lora",  # Path đến folder đã save
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

# Enable inference mode
FastLanguageModel.for_inference(model)

# Giờ có thể dùng để generate như bình thường!
"""

print("Code trên để load model đã save")
print("Copy và run trong notebook mới khi muốn dùng model")